In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import random
import numpy as np

In [ ]:
def to_sequence(data, seq_size=10):
    x, y = [], []

    for i in range(len(data)-seq_size):
        window = data[i:(i+seq_size)].values
        after_window = data[i+seq_size]
        x.append(window)
        y.append(after_window)

    return np.array(x), np.array(y)

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import *

vertical_MAE =[]
vertical_MSE=[]
vertical_RMSE =[]
vertical_MRE=[]
vertical_MMRE =[]
vertical_PRED=[]

for j in range(1,103):
  # Read csv file into a pandas dataframe
  df = pd.read_csv(f'drive/MyDrive/All patients/Data missing/5%/Patient_{j}.csv')
  # print(j)

  for i in range(len(df)):
    if np.isnan(df['SensorGLU'][i]):
      X, y = to_sequence(df[:i-1].SensorGLU)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1)
      # Create a SVR model
      model = SVR(kernel='rbf',gamma=0.5, C=10, epsilon = 0.05)
      # Train the model
      model.fit(X_train, y_train)
      # Make Predictions and Assess Model
      pred = model.predict(X_test)
      df['SensorGLU'][i]=pred



  vertical_MAE =np.append(vertical_MAE , mean_absolute_error(y_test,pred))
  vertical_MSE =np.append(vertical_MSE , mean_squared_error(y_test,pred))
  vertical_RMSE =np.append(vertical_RMSE , np.sqrt(mean_squared_error(y_test,pred)))
  vertical_MRE = np.append(vertical_MRE , np.abs((y_test-pred)/y_test)*100)
  vertical_MMRE = np.append(vertical_MMRE , np.mean(np.abs((y_test-pred)/y_test)*100))
  Pred = lambda p: len(vertical_MRE[vertical_MRE <= p])/len(vertical_MRE)*100
  PRED = Pred(25)
  vertical_PRED =np.append(vertical_PRED , PRED)

In [ ]:
import csv


test = pd.DataFrame({"MAE": vertical_MAE, "RMSE": vertical_RMSE, "MMRE": vertical_MMRE, "PRED": vertical_PRED})
test.to_csv(r'drive/MyDrive/All patients/Resultats/Patient_result_SVR_5.csv', index=False)

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from keras.layers import RNN, SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import *

vertical_MAE =[]
vertical_MSE=[]
vertical_RMSE =[]
vertical_MRE=[]
vertical_MMRE =[]
vertical_PRED=[]

for j in range(1,103):
  # Read csv file into a pandas dataframe
  df = pd.read_csv(f'drive/MyDrive/All patients/Data missing/5%/Patient_{j}.csv')

  for i in range(len(df)):
    if np.isnan(df['SensorGLU'][i]):
      X, y = to_sequence(df[:i-1].SensorGLU)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1)
      X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
      X_test = np.reshape(X_test, (X_test.shape[0],  X_test.shape[1],1))
      # Define the RNN model
      model = Sequential()
      model.add(SimpleRNN(64))
      model.add(Dense(1))
      # Compile the model
      model.compile(loss='mean_squared_error', optimizer='adam')
      # Train the model
      model.fit(X_train, y_train, epochs=200, verbose=0)
      # Make Predictions
      pred = model.predict(X_test)
      df['SensorGLU'][i]=pred


  vertical_MAE =np.append(vertical_MAE , mean_absolute_error(y_test,pred))
  vertical_MSE =np.append(vertical_MSE , mean_squared_error(y_test,pred))
  vertical_RMSE =np.append(vertical_RMSE , np.sqrt(mean_squared_error(y_test,pred)))
  vertical_MRE = np.append(vertical_MRE , np.abs((y_test-pred)/y_test)*100)
  vertical_MMRE = np.append(vertical_MMRE , np.mean(np.abs((y_test-pred)/y_test)*100))
  Pred = lambda p: len(vertical_MRE[vertical_MRE <= p])/len(vertical_MRE)*100
  PRED = Pred(25)
  vertical_PRED =np.append(vertical_PRED , PRED)


In [ ]:
import csv


test = pd.DataFrame({"MAE": vertical_MAE, "RMSE": vertical_RMSE, "MMRE": vertical_MMRE, "PRED": vertical_PRED})
test.to_csv(r'drive/MyDrive/All patients/Resultats/Patient_result_RNN_5.csv', index=False)